In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = '/home/gbotemi/.google/credential.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/02/20 19:34:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
df_green = spark.read.parquet('gs://new_york_city_trips_dataset/pq/green/*/*')

In [8]:
df_green.printSchema

<bound method DataFrame.printSchema of DataFrame[VendorID: int, lpep_pickup_datetime: timestamp, lpep_dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: int, PULocationID: int, DOLocationID: int, passenger_count: int, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, ehail_fee: double, improvement_surcharge: double, total_amount: double, payment_type: int, trip_type: int, congestion_surcharge: double]>

In [9]:
df_green = df_green \
            .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')\
            .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [10]:
df_yellow = spark.read.parquet('gs://new_york_city_trips_dataset/pq/yellow/*/*')

In [11]:
df_yellow = df_yellow \
            .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
            .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [12]:
df_yellow.count()

39649199

In [13]:
common_cols = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_cols.append(col)

In [14]:
common_cols

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [15]:
from pyspark.sql import functions as F

In [16]:
df_green_sel = df_green \
                    .select(common_cols) \
                    .withColumn('service_type', F.lit('green'))

In [17]:
df_yellow_sel = df_yellow \
                    .select(common_cols) \
                    .withColumn('service_type', F.lit('yellow'))

In [18]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [19]:
print(df_green.count())
print(df_yellow.count())
print(df_trips_data.count())

2304517


39649199


41953716


In [20]:
df_trips_data.summary()

DataFrame[summary: string, VendorID: string, store_and_fwd_flag: string, RatecodeID: string, PULocationID: string, DOLocationID: string, passenger_count: string, trip_distance: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string, payment_type: string, congestion_surcharge: string, service_type: string]

In [21]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [22]:
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

In [23]:
df_trips_data.registerTempTable('trips_data')


/home/gbotemi/spark/spark-3.2.3-bin-hadoop3.2/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [24]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [25]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [26]:
df_result.head()

Row(revenue_zone=250, revenue_month=datetime.datetime(2020, 2, 1, 0, 0), service_type='green', revenue_monthly_fare=15359.960000000061, revenue_monthly_extra=1282.5, revenue_monthly_mta_tax=117.5, revenue_monthly_tip_amount=56.01, revenue_monthly_tolls_amount=590.3200000000003, revenue_monthly_improvement_surcharge=180.0000000000011, revenue_monthly_total_amount=17598.43999999995, revenue_monthly_congestion_surcharge=11.0, avg_montly_passenger_count=1.2394957983193278, avg_montly_trip_distance=4.962810650887575)

In [27]:
df_result.coalesce(1).write.parquet('gs://new_york_city_trips_dataset/report/revenue/', mode='overwrite')